# **cdQA on Coronavirus articles on Russian language**

In [0]:
!pip install cdqa

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import re
import string
import os
import csv

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.converters import df2squad
from cdqa.pipeline import QAPipeline
import torch
import joblib
from cdqa.reader import BertProcessor, BertQA
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.utils.evaluation import evaluate_pipeline, evaluate_reader

/usr/local/lib/python3.6/dist-packages/cdqa/reader/bertqa_sklearn.py:35: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Preparing my data for QA-annotator

Для каждой статьи получаем её заголовок и делим текст на абзацы.
Всего 18 небольших файлов (статей).

In [0]:
directory = 'drive/My Drive/qa/'

In [0]:
titles = []
paragraphs = []

file_names = [i for i in range(1, 19)]
files = [ 'sample' + str(i) + '.txt' for i in file_names]

for file in files:
  f = open(directory + 'data/' + file, 'r')
  article = []
  for line in f:
    article.append(line)
  titles.append(article[0][:-1])
  text = ''
  for t in article[3::]:
    text+=t
  texts = text.split('\n\n')
  texts = [s.strip() for s in texts if s!='']
  paragraphs.append(texts)

In [0]:
len(titles), len(paragraphs)

(18, 18)

In [11]:
df = pd.DataFrame({'title': titles, 'paragraphs': paragraphs})
print("Len of table: ", len(df))
df.head()

Len of table:  18


,title,paragraphs
0,Часто задаваемые вопросы.,"[Коронавирусы – обширное семейство вирусов, ко..."
1,Как правильно носить маску?,[Маски могут иметь разную конструкцию. Они мог...
2,Что предпринять?,[Соблюдайте правила гигиены. Чтобы защитить се...
3,Мифы и ложные представления.,[Являются ли антибиотики эффективным средством...
4,Что означает находиться дома на карантине?,[Главное условие – не выходить из дома все 14 ...


In [0]:
df['paragraphs'][0]

['Коронавирусы – обширное семейство вирусов, которые поражают людей и животных. Известно, что некоторые из них способны вызывать у человека респираторные инфекции в диапазоне от обычной простуды до более серьезных состояний. Последний из недавно открытых коронавирусов вызывает заболевание COVID‑19. До вспышки инфекции в китайской провинции Ухань в декабре 2019 года о новом вирусе и заболевании ничего не было известно. Он передается главным образом воздушно-капельным путем в результате вдыхания капель, выделяемых из дыхательных путей больного, например при кашле или чихании, а также капель слюны или выделений из носа. Также он может распространяться, когда больной касается любой загрязненной поверхности, например дверной ручки. В этом случае заражение происходит при касании рта, носа или глаз грязными руками.',
 'Самое важное, что можно сделать, чтобы защитить себя, – это соблюдать правила личной гигиены и сократить посещения общественных и людных мест. Держите руки в чистоте, часто мой

In [12]:
df = filter_paragraphs(df)
df.head()

,title,paragraphs
0,Часто задаваемые вопросы.,"[Коронавирусы – обширное семейство вирусов, ко..."
1,Как правильно носить маску?,[Маски могут иметь разную конструкцию. Они мог...
2,Что предпринять?,[Соблюдайте правила гигиены. Чтобы защитить се...
3,Мифы и ложные представления.,[Являются ли антибиотики эффективным средством...
4,Общая информация о короновирусе.,[Симптомы и признаки коронавируса у человека. ...


In [0]:
len(df)

16

Сохраним наш dataframe в csv формате для будующей QA модели.

In [0]:
df.to_csv(directory+'data/dataframe.csv', index = False)

Сохраним наши данные в формате датасета SQuAD v1.1 в json файле.

In [0]:
json_data = df2squad(df=df, squad_version='v1.1', output_dir=directory+'data/', filename='dataset-coronavirus')

16it [00:00, 1946.09it/s]


## Training the Reader

Создадим reader на основе мультиязычного Bert и данных на русском языке.

In [0]:
train_processor = BertProcessor(bert_model = 'bert-base-multilingual-uncased', do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X=directory + 'data/sber_squad_clean-v1.1/train-v1.1.json')

Could not find answer: '' vs. 'Машина времени'
Could not find answer: '' vs. 'Старая крепость'
Could not find answer: '' vs. 'Everybody'
Could not find answer: '' vs. 'Георгики'
Could not find answer: '' vs. 'История Рима'
Could not find answer: '' vs. 'Бхагавадгита'
Could not find answer: '' vs. 'Машина времени'


In [0]:
 len(train_examples), len(train_features)

(45321, 47716)

In [0]:
ru_reader = BertQA(bert_model = 'bert-base-multilingual-uncased',
                   train_batch_size=10,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir=directory + 'models/ru_reader/')

ru_reader.fit(X=(train_examples, train_features))

BertQA(adam_epsilon=1e-08, bert_model='bert-base-multilingual-uncased',
       do_lower_case=True, fp16=False, gradient_accumulation_steps=1,
       learning_rate=3e-05, local_rank=-1, loss_scale=0, max_answer_length=30,
       n_best_size=20, no_cuda=False, null_score_diff_threshold=0.0,
       num_train_epochs=2, output_dir='drive/My Drive/qa/models/ru_reader/',
       predict_batch_size=8, seed=42, server_ip='', server_port='',
       train_batch_size=10, verbose_logging=False,
       version_2_with_negative=False, warmup_proportion=0.1, warmup_steps=0)

In [0]:
ru_reader.model.to('cpu')
ru_reader.device = torch.device('cpu')

In [0]:
joblib.dump(ru_reader, os.path.join(ru_reader.output_dir, 'mulilingbert_qa.joblib'))

['drive/My Drive/qa/models/ru_reader/mulilingbert_qa.joblib']

## Fine-tuning the reader

Дообучим наш Reader на наших данных о коронавирусе.

In [4]:
cdqa_pipeline = QAPipeline(reader='drive/My Drive/qa/models/ru_reader/mulilingbert_qa.joblib')

100%|██████████| 231508/231508 [00:00<00:00, 920453.50B/s]


In [8]:
cdqa_pipeline.fit_reader(directory + 'data/' + 'cdqa-ru-coronavirus-v1.1.json')

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='bert-base-multilingual-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=3e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir='drive/My Drive/qa/models/ru_reader/',
                         predict_...
                         server_port='', train_batch_size=10,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                  

In [0]:
cdqa_pipeline.dump_reader(directory + 'models/'+'bert_qa_ru_coronavirus.joblib')

## Instantiate the cdQA pipeline from a pre-trained reader model

Добавим в Retriever наши документы.

In [13]:
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='bert-base-multilingual-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=3e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir='drive/My Drive/qa/models/ru_reader/',
                         predict_...
                         server_port='', train_batch_size=10,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                  

## Evaluate model

In [14]:
evaluate_pipeline(cdqa_pipeline, directory + 'data/' + 'cdqa-ru-coronavirus-v1.1.json')



Evaluation results: {'exact_match': 5.314009661835748, 'f1': 14.788497069911307}


{'exact_match': 5.314009661835748, 'f1': 14.788497069911307}

In [15]:
evaluate_reader(cdqa_pipeline, directory + 'data/' + 'cdqa-ru-coronavirus-v1.1.json')

Evaluation expects v-1.1, but got dataset with v-v1.1


{'exact_match': 8.93719806763285, 'f1': 26.354934581472794}

Точность получилась довольно низкой. Проверим как модель ответит на вопросы.

## Making predictions

In [0]:
cdqa_pipeline.predict(query='Как передается коронавирус?')

('Воздушно-капельным путем',
 'Часто задаваемые вопросы.',
 'Как передается коронавирус? Воздушно-капельным путем (при кашле или чихании), контактным путем (поручни в транспорте, дверные ручки и другие загрязненные поверхности и предметы). Как и другие респираторные вирусы, коронавирус распространяется через капли, которые образуются, когда инфицированный человек кашляет или чихает. Кроме того, он может распространяться, когда инфицированный человек касается любой загрязненной поверхности, например, дверной ручки. Люди заражаются, когда они касаются загрязненными руками рта, носа или глаз.',
 4.793074290461236)

In [16]:
cdqa_pipeline.predict(query='Показания к госпитализации при коронавирусе?', n_predictions = 3)

[('читайте меньше новостей',
  'Бытовые вопросы.',
  'Лекарств нет, прививок тоже — как хотя бы тревожиться поменьше? Мы сделали специальную инструкцию как раз про это. Главный совет самый простой и одновременно самый трудный: если вам тревожно из-за новостей о коронавирусе, читайте меньше новостей. Также полезно найти время пообщаться с близкими, заняться спортом или хотя бы немного расслабиться.',
  2.496142509344046),
 ('беременность',
  'По-моему, меня неправильно лечат от коронавируса. Как быть? Кстати, а как лечить правильно?',
  'Но, когда эти симптомы появятся, не будет уже поздно? Нет. От появления симптомов до начала одышки и необходимости лечь в больницу обычно проходит 5–8 дней. После этого до развития тяжелого и действительно опасного состояния проходит еще пара дней.  У некоторых людей высок риск того, что заболевание будет прогрессировать быстро и примет тяжелую форму. По их поводу решения принимаются несколько иначе и индивидуально. В любом случае к их состоянию нужно б

In [0]:
query = 'Что отражает коэффициент выявления заразившихся?'
prediction = cdqa_pipeline.predict(query)

In [18]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: Что отражает коэффициент выявления заразившихся?
answer: полученными группами ученых Imperial Сollege London
title: Пик эпидемии в России придется на середину мая. Но тестирование нужно расширять и улучшать — иначе на этом пике можно застрять на несколько месяцев.
paragraph: «Коэффициент выявления» за время, прошедшее с предыдущего прогноза, резко вырос: если в конце марта в России выявляли около 20% инфицированных (с симптомами и без), то по данным на 17 апреля — около 45%. Наш коэффициент согласуется с данными, полученными группами ученых Imperial Сollege London (тут) и Университета Мельбурна (тут), которые представили свой прогноз для России (как и для десятков других стран).


In [22]:
cdqa_pipeline.predict(query='Как правильно мыть руки?')

('С мылом, не менее 20 секунд',
 'Бытовые вопросы.',
 'Обращаться за медицинской помощью нужно, если возникли тревожные симптомы: появилась одышка, боль в груди, спутанность сознания, посинели губы или лицо, трудно проснуться. В списке ВОЗ нет рекомендации носить маску, но о ней — разговор особый (чуть ниже). Ок, как правильно мыть руки? С мылом, не менее 20 секунд, не пропуская тыльной стороны ладоней. Мы подготовили подробную инструкцию и даже плакаты о том, как это делать, исполняя любимую песню. Пользуйтесь.',
 7.716528450772378)

In [25]:
cdqa_pipeline.predict(query='Что нужно для самодельного санитайзера?')

('Тестов нужно очень много',
 'Научные вопросы.',
 'Как тестируют на коронавирус? Есть три типа тестов: детекция генетического материала, тест на антитела и томография. Изо всех них почти всегда, когда речь идет о тестах, имеется в виду именно первый тип; делается он с помощью классической полимеразной цепной реакции — ПЦР. Разные страны преимущественно полагаются на тесты собственного производства, которые на самом деле не сильно отличаются друг от друга. Тестов нужно очень много.',
 2.377467736611582)

In [27]:
cdqa_pipeline.predict(query='Люди какого возраста наиболее уязвимы?')

('людям старше 65 лет',
 'Часто задаваемые вопросы.',
 'Кто в группе риска? Заразиться вирусом могут люди всех возрастов. Степень тяжести заболевания зависит от индивидуальных факторов. Однако людям старше 65 лет, а также гражданам с ослабленной иммунной системой и хроническими заболеваниями рекомендуется быть наиболее осторожными и внимательными к своему здоровью. Детям и внукам пожилых людей нужно позаботиться о старшем поколении, завезти продукты, лекарства, соблюдая все предосторожности, не контактируя с ними.',
 2.2798373993956864)

In [28]:
cdqa_pipeline.predict(query='Чем грипп похож на коронавирус?', n_predictions = 3)

[('Если у вас нет выраженных симптомов',
  'Бытовые вопросы.',
  'Как можно сдать тест на коронавирус? И нужно ли? Если у вас нет выраженных симптомов — нет, не нужно. Вне зависимости от результата теста, вам все равно предстоит соблюдать карантин и находиться дома. Обращаться за медпомощью следует только при повышении температуры, появлении кашля и затруднении дыхания. Вот здесь об этом написано подробнее.',
  2.8524385507728165),
 ('распространяется через капли',
  'Часто задаваемые вопросы.',
  'Как передается коронавирус? Воздушно-капельным путем (при кашле или чихании), контактным путем (поручни в транспорте, дверные ручки и другие загрязненные поверхности и предметы). Как и другие респираторные вирусы, коронавирус распространяется через капли, которые образуются, когда инфицированный человек кашляет или чихает. Кроме того, он может распространяться, когда инфицированный человек касается любой загрязненной поверхности, например, дверной ручки. Люди заражаются, когда они касаются з

In [29]:
cdqa_pipeline.predict(query='От чего зависит цена маски?', n_predictions = 3)

[('придется ли вводить карантин снова',
  'Пик эпидемии в России придется на середину мая. Но тестирование нужно расширять и улучшать — иначе на этом пике можно застрять на несколько месяцев.',
  'Владимир Путин 28 апреля сказал, что пик эпидемии в России еще не пройден, но пообещал, что с 12 мая в некоторых регионах начнут постепенно отменять ограничительные меры. Как показывает моделирование «Медузы», первая волна эпидемии коронавируса в стране действительно приближается к перелому. В середине мая, скорее всего, «пик» эпидемии будет пройден: каждый день число вновь заразившихся будет меньше, чем число выздоровевших. Тогда остро встанет вопрос, как и когда отменить ограничительные меры: нельзя спешить, потому что спешка грозит новыми вспышками эпидемии; но нельзя и долго ждать, потому что ограничения разрушают экономику. От того, как будет решена дилемма, зависит, придется ли вводить карантин снова — для борьбы со второй волной эпидемии.',
  3.2139932601470385),
 ('из-за различной про

In [30]:
cdqa_pipeline.predict(query='Что предпринять чтобы защитить себя от заражения коронавирусом?', n_predictions = 3)

[('Заразиться новым коронавирусом',
  'Мифы и ложные представления.',
  'Правда ли, что новым коронавирусом могут заразиться только пожилые люди, или молодежь тоже восприимчива к этой инфекции? Заразиться новым коронавирусом (COVID-19) могут представители всех возрастных категорий. Как представляется, пожилые люди и люди, больные определенными заболеваниями (например, астмой, диабетом, болезнью сердца), подвержены повышенному риску развития тяжелых форм коронавирусной инфекции. ВОЗ рекомендует лицам любого возраста принимать меры по защите от заражения, например посредством соблюдения гигиены рук и кашлевой гигиены.',
  4.369935346182076),
 ('крайне важно правильно ее носить',
  'Как правильно носить маску?',
  'Маски могут иметь разную конструкцию. Они могут быть одноразовыми или могут применяться многократно. Есть маски, которые служат 2, 4, 6 часов. Стоимость этих масок различная, из-за различной пропитки. Но нельзя все время носить одну и ту же маску, тем самым вы можете инфицирова

In [31]:
cdqa_pipeline.predict(query='Лучший способ борьбы с эпидемией?', n_predictions = 3)

[('ограничительные меры',
  'Пик эпидемии в России придется на середину мая. Но тестирование нужно расширять и улучшать — иначе на этом пике можно застрять на несколько месяцев.',
  'Точки — натуральный логарифм отношения зарегистрированных случаев и выздоровевших/умерших (то есть количества случаев за 15 дней до того) за каждый день — лежат (с отклонениями) на прямой линии, представляющей наилучшее линейное приближение. Все параметры эпидемии и борьбы с ней —ограничительные меры, изменение эффективности тестирования, коэффициент воспроизводства в разные моменты времени — «записаны» в наклон функции (стандартное отклонение распределения данных).',
  3.152241968604963),
 ('среднего',
  'Что происходит в других странах?',
  'При таком сравнении стран становится понятно, что на самом деле Германия не сильно отличается по динамике от Италии или Франции — скорее находится на более раннем этапе своей эпидемии. Есть на этом графике и те страны и территории, которые справляются явно лучше сред

## Итог. Получение ответа.

Получение ответа в одной функции. Вместо 1 ответа можно получить несколько наиболее релевантных, передав в функцию get_answer вместе с вопросом число n_predictions, если вы не хотите получить дополнительную информацию об ответе, передайте flag = False.

In [0]:
def get_answer(query, n_predictions = 1, flag = True):
  def returnRightList(my_df):
    for i, k in enumerate(my_df['paragraphs']):
      sent_list = re.split(r'\',', k[1:-1])
      sent_list = [s.strip() for s in sent_list]
      sent_list = [s.strip("\'") for s in sent_list]
      my_df['paragraphs'][i] = sent_list
    
    return my_df

  df = pd.read_csv('drive/My Drive/qa/data/dataframe.csv', sep=',')
  df = returnRightList(df)

  cdqa_pipeline = QAPipeline(reader='drive/My Drive/qa/models/bert_qa_ru_coronavirus.joblib')
  cdqa_pipeline.fit_retriever(df=df)
  if n_predictions>1:
    predictions = cdqa_pipeline.predict(query, n_predictions)
    answers = []
    for prediction in predictions:
      answers.append(prediction[0])
      if flag:
        print('Вопрос: {}'.format(query))
        print('Ответ: {}'.format(prediction[0]))
        print('Заголовок документа: {}'.format(prediction[1]))
        print('Абзац: {}'.format(prediction[2]))
        print()
    return answers
  else:
    prediction = cdqa_pipeline.predict(query)
    if flag:
      print('Вопрос: {}'.format(query))
      print('Ответ: {}'.format(prediction[0]))
      print('Заголовок документа: {}'.format(prediction[1]))
      print('Абзац: {}'.format(prediction[2]))
      print()
    return prediction[0]

In [48]:
get_answer('Что может помочь уменьшить волну эпидемии?', n_predictions=3, flag=False)

['% населения', 'пройдет пик', 'не прибегая к новым карантинам']

In [49]:
get_answer('Редкие симптомы коронавируса?', n_predictions=3, flag=False)

['высокая температура тела, кашель', 'человека', 'Нет']

In [45]:
get_answer('Что такое пандемия?')

Вопрос: Что такое пандемия?
Ответ: это просто глобальная эпидемия
Заголовок документа: Научные вопросы.
Абзац: Что за слово такое — пандемия? Пандемия — это просто глобальная эпидемия. А эпидемия — это резкое превышение заболеваемости над ожидаемым многолетним уровнем. Строгого численного определения у этих понятий нет, как нет никакой магии и в самом понятии пандемии. Для ВОЗ объявление вспышки коронавируса пандемией — это скорее политический жест, а не научное открытие. Главное, что при этом происходит, — признание того, что сдержать инфекцию локальными мерами не удалось и что для дальнейшей борьбы требуются усилия многих стран.



'это просто глобальная эпидемия'

In [51]:
get_answer('Коронавирус опаснее гриппа?', n_predictions=3)

Вопрос: Коронавирус опаснее гриппа?
Ответ: составляет 3 дня, а у COVID-19 – 5-6 дней
Заголовок документа: Часто задаваемые вопросы.
Абзац: В чем разница между коронавирусом и вирусом гриппа? Коронавирус и вирус гриппа похожи основными симптомами, способами передачи и профилактики, но имеют и принципиальные различия. Например, инкубационный период: у гриппа он составляет 3 дня, а у COVID-19 – 5-6 дней. Также коронавирусная инфекция вызывает больше тяжелых и критических случаев, что, в свою очередь, приводит к более высоким показателям смертности, чем у сезонного гриппа.

Вопрос: Коронавирус опаснее гриппа?
Ответ: раз опаснее сезонного гриппа
Заголовок документа: Научные вопросы.
Абзац: Какая реальная смертность у этого заболевания? Говорят, оно не страшнее гриппа? Летальность COVID-19 уже довольно хорошо известна и составляет около 1% или чуть меньше. Точное значение зависит от возрастной структуры конкретной популяции и уровня медицинской помощи в стране. О том, как правильно считать л

['составляет 3 дня, а у COVID-19 – 5-6 дней',
 'раз опаснее сезонного гриппа',
 'в клетках гусениц']

In [53]:
get_answer('Сколько занимает разработка вакцины?')

Вопрос: Сколько занимает разработка вакцины?
Ответ: сама вакцинация — то есть год или два
Заголовок документа: Научные вопросы.
Абзац: Однако в невакцинированной популяции вторичная вспышка может все еще случиться в любой момент. В худшем случае карантин придется держать столько времени, сколько займут разработка вакцины и сама вакцинация — то есть год или два. В этом случае речь будет идти, скорее всего, не о полном локдауне на два года, а об отдельных мерах вроде перевода школьников на удаленное обучение или введении особых систем слежения за потенциальными носителями.



'сама вакцинация — то есть год или два'